In [1]:
#import dash_core_components as dcc replace with from dash import dcc 
import pandas as pd
from dash import dcc
import dash_core_components as dcc
# replace import dash_html_components as html with #from dash import html
from dash import html
from jupyter_dash import JupyterDash 
from dash.dependencies import Input, Output
import dash_html_components as html
import plotly.express as px

C:\Users\andhi\AppData\Local\Temp\ipykernel_5092\1872653772.py:4: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\andhi\AppData\Local\Temp\ipykernel_5092\1872653772.py:9: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [2]:
from dash import Dash, dcc, html
import os

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = JupyterDash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                            options=[
                                                {'label': 'All Sites', 'value': 'ALL'},
                                                {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                                {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                ],
                                            value='ALL',    
                                            placeholder='Select a Launch Site here',
                                            searchable=True
                                            ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider',
                                    min=0, max=10000, step=1000,
                                    marks={0: '0',
                                        1000:'1000', 2000:'2000', 3000:'3000', 4000:'4000', 5000:'5000', 6000:'6000',7000:'7000', 8000:'8000',9000:'9000', 10000:'10000'},
                                    value=[min_payload, max_payload]),
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    filtered_df1 = spacex_df
    if entered_site == 'ALL':
        filtered_df1=filtered_df1.groupby(['Launch Site','class']).size().reset_index(name='class count')
        fig1 = px.pie(filtered_df1, values='class count',names='class',title="Total Success Launches for All Sites")
        return fig1
    else:
        filtered_df2 = spacex_df[spacex_df['Launch Site'] == entered_site]
        filtered_df2 =filtered_df2.groupby(['Launch Site','class']).size().reset_index(name='class count')
        fig2=px.pie(filtered_df2,values='class count',names='class',title=f"Total Success Launches for site {entered_site}")
        return fig2
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),
              Input(component_id='payload-slider', component_property='value')]
             )
def get_scatter_chart(drop_selection,slider):
    filtered_data1 = spacex_df[spacex_df['Payload Mass (kg)']>=slider[0] & (spacex_df['Payload Mass (kg)'] <= slider[1])]
    if drop_selection == 'ALL':
        
        scat = px.scatter(filtered_data1,x='Payload Mass (kg)',y='class',color = "Booster Version Category",\
                          title="Relationship between Payload and Success Rate") #scatterchart
        return scat
    else:
        filtered_data2= filtered_data1[filtered_data1['Launch Site']== drop_selection]        
        scat= px.scatter(filtered_data2,x='Payload Mass (kg)', y='class',color = "Booster Version Category",\
                        title=f"Relationship between Payload and Success Rate for {drop_selection}")
        return scat
# Run the app
if __name__ == '__main__':
    app.run_server(mode='jupyterlab', port = 8090 ,dev_tools_ui=True, dev_tools_hot_reload =True, threaded=True)

In [5]:
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [8]:
filtered_df2 =spacex_df.groupby(['Launch Site','class']).size().reset_index(name='class count')
filtered_df2.head()

,Launch Site,class,class count
0,CCAFS LC-40,0,19
1,CCAFS LC-40,1,7
2,CCAFS SLC-40,0,4
3,CCAFS SLC-40,1,3
4,KSC LC-39A,0,3
